<a href="https://colab.research.google.com/github/vpavankalyan/FER/blob/main/fer_using_vgg_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_applications

     |████████████████████████████████| 50 kB 1.6 MB/s eta 0:00:011


In [ ]:
!pip install keras_vggface

In [ ]:
from keras_vggface.vggface import VGGFace

In [ ]:
image_size=224
vgg_conv = VGGFace(weights='vggface', include_top=False, input_shape=(image_size, image_size, 3))

58916864/58909280 [==============================] - 1s 0us/step


In [ ]:
for layer in vgg_conv.layers:
    layer.trainable = False

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(7, activation='softmax'))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
train_generator = image_gen.flow_from_directory('../input/face-expression-recognition-dataset/images/train',
        target_size=(image_size, image_size),
        batch_size=80,
        class_mode='categorical')

Found 28821 images belonging to 7 classes.


In [ ]:
validation_generator = image_gen.flow_from_directory('../input/face-expression-recognition-dataset/images/validation',
        target_size=(image_size, image_size),
        batch_size=30,
        class_mode='categorical',
        shuffle=False)

Found 7066 images belonging to 7 classes.


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=0.0005),
              metrics=['acc'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)

Epoch 1/30
360/360 [==============================] - 657s 2s/step - loss: 1.4814 - acc: 0.4258 - val_loss: 1.1819 - val_acc: 0.5587
Epoch 2/30
360/360 [==============================] - 446s 1s/step - loss: 1.2222 - acc: 0.5393 - val_loss: 1.1331 - val_acc: 0.5753
Epoch 3/30
360/360 [==============================] - 457s 1s/step - loss: 1.1595 - acc: 0.5690 - val_loss: 1.1289 - val_acc: 0.5839
Epoch 4/30
360/360 [==============================] - 462s 1s/step - loss: 1.1379 - acc: 0.5749 - val_loss: 1.1188 - val_acc: 0.5838
Epoch 5/30
360/360 [==============================] - 440s 1s/step - loss: 1.1164 - acc: 0.5873 - val_loss: 1.0696 - val_acc: 0.6035
Epoch 6/30
360/360 [==============================] - 424s 1s/step - loss: 1.0918 - acc: 0.5959 - val_loss: 1.0795 - val_acc: 0.5982
Epoch 7/30
360/360 [==============================] - 426s 1s/step - loss: 1.0828 - acc: 0.5960 - val_loss: 1.0711 - val_acc: 0.6070
Epoch 8/30
360/360 [==============================] - 425s 1s/step - 

In [ ]:
model.save("vgg_face1.h5")